In [1]:
import sys
import pathlib
module_path = str(pathlib.Path().absolute()).replace('/notebooks', '')
sys.path.append(module_path)

In [ ]:
from scripts.jobs.dataset_creation import ReadRasterFile, CreateCompositeStridedArray, PreProcessBorderRaster, StoreCompositeDataHDF5, PreProcessPopulationRaster

raster_dem_fn = '/mnt/share/mnt/RESEARCH/SATELLITE/WORK/dem_aggr_rst.tif'
raster_pop_fn = '/mnt/share/mnt/RESEARCH/SATELLITE/WORK/pop_rst.tif'
raster_nuts_fn = '/mnt/share/mnt/RESEARCH/SATELLITE/WORK/nuts_rst.tif'

rasters = {
    'pop': {
        'type': 'input',
        'data': PreProcessPopulationRaster(ReadRasterFile(raster_pop_fn)),
        'bad_value_threshold': -1000
    },
    'nuts': {
        'type': 'output',
        'data': PreProcessBorderRaster(ReadRasterFile(raster_nuts_fn)),
        'bad_value_threshold': -1000
    }
}
(train_x, train_y), (test_x, test_y) = CreateCompositeStridedArray(rasters)

for i in range(2):
    selection_mask = [(x == i)[0] for x in train_y]
    StoreCompositeDataHDF5(train_x[selection_mask], train_y[selection_mask], 'pop_nuts_train_' + str(i))
    selection_mask = [(x == i)[0] for x in test_y]
    StoreCompositeDataHDF5(test_x[selection_mask], test_y[selection_mask], 'pop_nuts_test_' + str(i))


In [2]:

from scripts.jobs.dataset_creation import CreateTFDatasetFromCompositeGenerator
from scripts.jobs.networks.conv_classifier import TrainConvClassifierModel

train_dataset = CreateTFDatasetFromCompositeGenerator('pop_nuts_train', 2)
test_dataset = CreateTFDatasetFromCompositeGenerator('pop_nuts_test', 2)
model, history = TrainConvClassifierModel(train_dataset, test_dataset)
model.save('/mnt/share/mnt/RESEARCH/SATELLITE/WORK/pop_nuts_model') 


Epoch 1/10
100/100 [==============================] - 21s 215ms/step - loss: 24.7894 - mae: 0.2030 - mse: 0.1665 - accuracy: 0.8217 - val_loss: 0.3539 - val_mae: 0.1066 - val_mse: 0.0540 - val_accuracy: 0.9328
Epoch 2/10
100/100 [==============================] - 20s 199ms/step - loss: 0.2095 - mae: 0.0968 - mse: 0.0485 - accuracy: 0.9384 - val_loss: 0.1349 - val_mae: 0.0982 - val_mse: 0.0478 - val_accuracy: 0.9375
Epoch 3/10
100/100 [==============================] - 33s 326ms/step - loss: 0.1384 - mae: 0.0903 - mse: 0.0442 - accuracy: 0.9434 - val_loss: 0.1316 - val_mae: 0.0958 - val_mse: 0.0467 - val_accuracy: 0.9375
Epoch 4/10
100/100 [==============================] - 30s 296ms/step - loss: 0.1217 - mae: 0.0886 - mse: 0.0432 - accuracy: 0.9414 - val_loss: 0.1274 - val_mae: 0.0928 - val_mse: 0.0451 - val_accuracy: 0.9406
Epoch 5/10
100/100 [==============================] - 24s 238ms/step - loss: 0.1140 - mae: 0.0832 - mse: 0.0403 - accuracy: 0.9480 - val_loss: 0.1258 - val_mae: 0.

In [4]:
import tensorflow as tf
from scripts.jobs.dataset_creation import ReadRasterFile, PreProcessPopulationRaster
from scripts.jobs.model_prediction import PredictClassifierRaster, WriteResultRaster

raster_fn = '/mnt/share/mnt/RESEARCH/SATELLITE/WORK/pop_rst.tif'
model = tf.keras.models.load_model('/mnt/share/mnt/RESEARCH/SATELLITE/WORK/pop_nuts_model')

a = PreProcessPopulationRaster(ReadRasterFile(raster_fn))

data = PredictClassifierRaster(a, model)

out_rst_fn = '/mnt/share/mnt/RESEARCH/SATELLITE/WORK/result_pop_nuts.tif'
WriteResultRaster(data, raster_fn, out_rst_fn, channels=0)

In [ ]:
from scripts.jobs.dataset_creation import ReadRasterFile, CreateCompositeStridedArray, PreProcessBorderRaster, StoreCompositeDataHDF5
from matplotlib import pyplot

raster_dem_fn = '/mnt/share/mnt/RESEARCH/SATELLITE/WORK/dem_aggr_rst.tif'
raster_nuts_fn = '/mnt/share/mnt/RESEARCH/SATELLITE/WORK/pop_rst.tif'

pyplot.imshow(ReadRasterFile(raster_dem_fn), cmap='viridis')
pyplot.show()
pyplot.imshow(ReadRasterFile(raster_nuts_fn), cmap='viridis')
pyplot.show()

In [ ]:
a = ReadRasterFile('/mnt/share/mnt/RESEARCH/SATELLITE/WORK/pop_rst.tif')
a[a < -10] = -10
a[(a < 3) & (a > -10)] = 0
a[a > 2] = 1
pyplot.hist(a, bins='auto') 
pyplot.show()